In [ ]:
!pip install -U pytorch-lightning==1.5.9 neptune-client albumentations opencv-python segmentation-models-pytorch

     |████████████████████████████████| 527 kB 5.6 MB/s 
     |████████████████████████████████| 301 kB 32.2 MB/s 
     |████████████████████████████████| 102 kB 33.0 MB/s 
     |████████████████████████████████| 60.4 MB 43 kB/s 
     |████████████████████████████████| 88 kB 3.7 MB/s 
     |████████████████████████████████| 596 kB 34.9 MB/s 
     |████████████████████████████████| 133 kB 23.6 MB/s 
     |████████████████████████████████| 397 kB 29.8 MB/s 
     |████████████████████████████████| 952 kB 41.0 MB/s 
     |████████████████████████████████| 829 kB 39.6 MB/s 
     |████████████████████████████████| 1.1 MB 37.4 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 181 kB 43.0 MB/s 
     |████████████████████████████████| 132 kB 34.6 MB/s 
     |████████████████████████████████| 8.5 MB 37.1 MB/s 
     |████████████████████████████████| 79 kB 8.0 MB/s 
     |████████████████████████████████| 127 kB 44.1 MB/s 
     |████████████████

In [ ]:
from albumentations.augmentations.transforms import PadIfNeeded
import math
import random
import torch
import cv2
import numpy as np
from tqdm import tqdm
from pathlib import Path
from typing import List
from albumentations.pytorch.transforms import ToTensorV2
import albumentations as A
import torch.nn as nn
import pytorch_lightning as pl
import torchmetrics
from segmentation_models_pytorch import Unet

Wgranie danych 

In [ ]:
!wget "https://chmura.put.poznan.pl/s/MLk1k6RWWQQuOXs/download?path=%2F&files=train.tar.xz" -O train.tar.xz
!tar xf train.tar.xz
!rm train.tar.xz

--2022-02-21 16:32:32--  https://chmura.put.poznan.pl/s/MLk1k6RWWQQuOXs/download?path=%2F&files=train.tar.xz
Resolving chmura.put.poznan.pl (chmura.put.poznan.pl)... 150.254.5.31, 2001:808:201::5:31
Connecting to chmura.put.poznan.pl (chmura.put.poznan.pl)|150.254.5.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 251078400 (239M) [application/octet-stream]
Saving to: ‘train.tar.xz’

train.tar.xz        100%[===================>] 239.45M  21.2MB/s    in 13s     

2022-02-21 16:32:45 (18.7 MB/s) - ‘train.tar.xz’ saved [251078400/251078400]



Przydatne funkcje do konwersji

In [ ]:
def convert_rgb_to_ids(labels: np.ndarray) -> np.ndarray:
  result = np.zeros(labels.shape[:2], dtype=np.uint8)
  result[np.where((labels == (0, 0, 255)).all(axis=2))] = 1
  result[np.where((labels == (0, 255, 0)).all(axis=2))] = 2
  result[np.where((labels == (255, 0, 0)).all(axis=2))] = 3
  return result


def convert_ids_to_rgb(labels: np.ndarray) -> np.ndarray:
  result = np.zeros((*labels.shape, 3), dtype=np.uint8)
  result[labels == 1] = (0, 0, 255)
  result[labels == 2] = (0, 255, 0)
  result[labels == 3] = (255, 0, 0)
  return result

Customowy obiekt dataset dziedziczony z torcha

In [ ]:
class LunarDataset(torch.utils.data.Dataset):
  def __init__(self, path: Path, file_names: List[str], augment: bool = False):
    self._file_names = file_names
    self._images_dir = path / 'images'
    self._labels_dir = path / 'masks'
    self._augment = augment

    self.image_size = (270, 480)
    self.padded_image_size = (
        math.ceil(self.image_size[0] / 32) * 32,
        math.ceil(self.image_size[1] / 32) * 32
    )

    self.transforms = A.Compose([
      A.Resize(*self.image_size),
      A.PadIfNeeded(*self.padded_image_size),
      A.ToFloat(max_value=255),
      ToTensorV2()
    ])
    self.augmentations = A.Compose([
      A.Resize(*self.image_size),
      A.PadIfNeeded(*self.padded_image_size),
      A.ToFloat(max_value=255),
      ToTensorV2()
    ])

  def __getitem__(self, index: int):
    image_path = self._images_dir / self._file_names[index].replace('.png', '.jpg')
    labels_path = self._labels_dir / self._file_names[index]

    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    labels = cv2.imread(str(labels_path))
    labels = cv2.cvtColor(labels, cv2.COLOR_BGR2RGB)
    labels = convert_rgb_to_ids(labels)

    if self._augment:
      transformed = self.augmentations(image=image, mask=labels)
    else:
      transformed = self.transforms(image=image, mask=labels)

    return transformed['image'], transformed['mask'].type(torch.int64)

  def __len__(self):
    return len(self._file_names)

  def _convert_mask(self, mask):
    # (h, w, 3) -> (h, w)
    result = np.zeros(mask.shape[:2], dtype=np.uint8)
    result[np.where((mask == (0, 0, 255)).all(axis=2))] = 1
    result[np.where((mask == (0, 255, 0)).all(axis=2))] = 2
    result[np.where((mask == (255, 0, 0)).all(axis=2))] = 3
    
    return result

  def calculate_weights(self):
    classes_sum = np.zeros((4,), dtype=np.uint64) 
    for file_name in tqdm(self._file_names):
      labels_path = self._labels_dir / file_name
      labels = cv2.imread(str(labels_path))
      labels = cv2.cvtColor(labels, cv2.COLOR_BGR2RGB)
      labels = self._convert_mask(labels)
      
      histogram, _ = np.histogram(labels.flatten(), bins=4, range=(0, 4))
      classes_sum += histogram.astype(np.uint64)
    
    weights = 1 / classes_sum
    weights /= np.sum(weights)
    return weights

Wczytywanie i dzielenie danych treningowych 

In [ ]:
from sklearn.model_selection import train_test_split

base_path = Path('/content/LunarSeg/train')
train_names = sorted([path.name for path in (base_path / 'masks').iterdir()])

train_names, val_names = train_test_split(train_names, test_size=0.15, random_state=42)

train_dataset = LunarDataset(base_path, train_names, augment=True)
val_dataset = LunarDataset(base_path, val_names)

Próba loss function z odwróceniem wag

In [ ]:
train_dataset.calculate_weights()

In [ ]:
weights = np.array([0.07004456, 0.02316237, 0.66572621, 0.24106686], dtype=np.float32)

Próba loss function z dice loss

In [ ]:
class MultiClassDiceLoss(nn.Module):
  def __init__(self, smooth = 1.0):
    super().__init__()

    self._smooth = smooth

  def forward(self, preds, ground_truth):
    preds = torch.softmax(preds, dim=1)

    num_classes = preds.shape[1]
    dice_sum = torch.tensor(0.0, dtype=torch.float32, device=preds.device)
    for class_id in range(num_classes):
      class_preds = preds[:, class_id].reshape(-1)
      class_ground_truth = (ground_truth == class_id).view(-1)

      tp = (class_preds * class_ground_truth).sum()
      class_dice = 1 - (2 * tp + self._smooth) / (class_preds.sum() + class_ground_truth.sum() + self._smooth)
      dice_sum += class_dice

    return dice_sum / num_classes

Segment class

In [ ]:
class Segmenter(pl.LightningModule):
    def __init__(self):
        super().__init__()
        
        self.network = Unet(encoder_name='resnet50', classes=4)

        # self.loss_function = torch.nn.CrossEntropyLoss()
        self.loss_function = MultiClassDiceLoss()
        # self.loss_function = nn.CrossEntropyLoss(
            # weight=torch.from_numpy(weights)
        # )
        
        metrics = torchmetrics.MetricCollection([
            torchmetrics.Precision(num_classes=4, average='macro', mdmc_average='samplewise'),
            torchmetrics.Recall(num_classes=4, average='macro', mdmc_average='samplewise'),
            torchmetrics.F1Score(num_classes=4, average='macro', mdmc_average='samplewise'),
            torchmetrics.Accuracy(num_classes=4, average='macro', mdmc_average='samplewise')
        ])
        self.train_metrics = metrics.clone('train_')
        self.val_metrics = metrics.clone('val_')

    def forward(self, x):
        return self.network(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.loss_function(outputs, labels)
        self.log('train_loss', loss)

        outputs = torch.softmax(outputs, dim=1)
        self.log_dict(self.train_metrics(outputs, labels))
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        
        loss = self.loss_function(outputs, labels)
        self.log('val_loss', loss, prog_bar=True)

        outputs = torch.softmax(outputs, dim=1)
        self.log_dict(self.val_metrics(outputs, labels))

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)





In [ ]:
segmenter = Segmenter()

model_checkpoint = pl.callbacks.ModelCheckpoint(dirpath='/content/checkpoints')
early_stopping = pl.callbacks.EarlyStopping(monitor='val_loss', patience=10)
logger = pl.loggers.NeptuneLogger(
    api_key='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzOWI2ZGJmZi1hNTVjLTQ4NmQtODBmOS00MDdkYWMyM2JhOGYifQ==',
    project='LunarSeg'
)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8, num_workers=2)

trainer = pl.Trainer(logger=logger, callbacks=[model_checkpoint, early_stopping], gpus=1, max_epochs=100)
trainer.fit(segmenter, train_dataloaders=train_loader, val_dataloaders=val_loader)

logger.run.stop()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | network       | Unet               | 32.5 M
1 | loss_function | MultiClassDiceLoss | 0     
2 | train_metrics | MetricCollection   | 0     
3 | val_metrics   | MetricCollection   | 0     
-----------------------------------------------------
32.5 M    Trainable params
0         Non-trainable params
32.5 M    Total params
130.086   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /content/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

https://app.neptune.ai/jumpincrane/LunarSeg/e/LUN-6
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 9 operations to synchronize with Neptune. Do not kill this process.


All 9 operations synced, thanks for waiting!


Wgranie danych do predykcji

In [ ]:
!wget "https://chmura.put.poznan.pl/s/MLk1k6RWWQQuOXs/download?path=%2F&files=test.tar.xz" -O test.tar.xz
!tar xf test.tar.xz
!rm test.tar.xz

--2022-02-21 20:31:02--  https://chmura.put.poznan.pl/s/MLk1k6RWWQQuOXs/download?path=%2F&files=test.tar.xz
Resolving chmura.put.poznan.pl (chmura.put.poznan.pl)... 150.254.5.31, 2001:808:201::5:31
Connecting to chmura.put.poznan.pl (chmura.put.poznan.pl)|150.254.5.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221311228 (211M) [application/octet-stream]
Saving to: ‘test.tar.xz’

test.tar.xz         100%[===================>] 211.06M  22.3MB/s    in 11s     

2022-02-21 20:31:14 (18.9 MB/s) - ‘test.tar.xz’ saved [221311228/221311228]



In [ ]:
device = torch.device('cuda')

In [ ]:
segmenter = Segmenter.load_from_checkpoint(model_checkpoint.best_model_path).to(device)  # wczytanie najlepszych wag z treningu
segmenter = segmenter.eval()

In [ ]:
import cv2

input_transforms = val_dataset.transforms
output_transforms = A.Compose([
  A.CenterCrop(*val_dataset.image_size),
  A.Resize(720, 1280, interpolation=cv2.INTER_NEAREST)
])

test_base_path = Path('/content/LunarSeg/test')
predictions_path = Path('/content/LunarSeg/test/predictions')
predictions_path.mkdir(exist_ok=True, parents=True)

for test_image_path in (test_base_path / 'images').iterdir():
  image = cv2.imread(str(test_image_path))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = input_transforms(image=image)['image'][None, ...]

  with torch.no_grad():
    prediction = segmenter(image.to(device)).cpu().squeeze().argmax(dim=0).numpy()
  
  prediction = convert_ids_to_rgb(prediction)
  prediction = cv2.cvtColor(prediction, cv2.COLOR_RGB2BGR)
  prediction = output_transforms(image=prediction)['image']

  cv2.imwrite(str(predictions_path / f'{test_image_path.stem}.png'), prediction)

In [ ]:
# predictions_path = Path('/content/LunarSeg/test/predictions_reverse_weights')
predictions_path = Path('/content/LunarSeg/test/predictions_dice')
# predictions_path = Path('/content/drive/MyDrive/Automatyka/mgr/predictions')

In [ ]:
import requests
import pickle
import zlib

from multiprocessing.dummy import Pool as ThreadPool

sum_result = 0

def calculate_score(prediction_path: Path):
  prediction = cv2.imread(str(prediction_path))
  prediction = cv2.cvtColor(prediction, cv2.COLOR_BGR2RGB)

  response = requests.post(f'http://zpo.dpieczynski.pl/{prediction_path.stem}', data=zlib.compress(pickle.dumps(prediction)))
  if response.status_code == 200:
    result = response.json()
    global sum_result
    sum_result += float(str(result)[6:-1])
    return f'{prediction_path.name} {result}'
  else:
    return f'Error processing prediction {prediction_path.name}: {response.text}'
  
  return None

i = 0
with ThreadPool(processes=16) as pool:
  for result in pool.imap_unordered(calculate_score, predictions_path.iterdir()):
    i += 1
print(i)

print(sum_result / i)


473
0.8765562945390101


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r '/content/LunarSeg/test/predictions_reverse_weights' '/content/drive/MyDrive/Automatyka/mgr/predictions_reverse_weights'